# Разновидности куч

Для куч свойственны основные операции:
* add
* getMax
* remove

Но зачастую возникает необходимость иметь дополнительные операции. Например операции быстрого объединения двух куч, поэтому существует такое большое количество их разновидностей.

### Эйлеров обход

Виды обхода:
* рекурсивный
* циклический

Замена рекурсии на цикл. На примере деструктора.

## Бинарная куча (дерево)

## Биноминальная куча (Binomial Heap)

Описание узла

In [1]:
template <typename T>
struct BinomialNode
{
    float key;
    int degree;
    BinomialNode *child, *sibling, *parent;
    T data;
};

Интерфейс кучи

In [2]:
template <typename T>
class BinomialHeap {
public:
    BinomialNode<T> root;
    T invalid;

    BinomialHeap(T invalidValue);
    virtual ~BinomialHeap();

    void insert(T data, float key = 0.0);
    T getMinimum(T data);
    T extractMin();
    void merge(BinomialHeap<T>* heap);

    void decreaseKey(BinomialNode<T>* node, float key);
    void remove(BinomialNode<T>* node);
    };

Создание нового узла

In [3]:
template <typename T>
BinomialNode<T>* newNode(){
    auto result = new BinomialNode<T>;
    result->sibling = nullptr;
    result->child = nullptr;
    result->parent = nullptr;
    result->degree = 1;
    result->key = 0.0;
    return result;
}

Разновидность создания

In [4]:
template <typename T>
BinomialNode<T>* newNodeInit(T data, float key = 0.0){
    BinomialNode<T>* result = newNode<T>();
    result->data = data;
    result->key = key;
    return result;
}

Слияние куч

In [5]:
template <typename T>
BinomialNode<T>* mergeBinomialHeaps(BinomialNode<T>* H1Root, BinomialNode<T>* H2Root)  {
    if (H1Root == nullptr)
        return H2Root;
    if (H2Root == nullptr)
        return H1Root;

    BinomialNode<T> result;
    BinomialNode<T>* curRes = &result;
    BinomialNode<T>* curH1 = H1Root;
    BinomialNode<T>* curH2 = H2Root;

    // вставка неповторяющихся куч
    while ((curH1 != nullptr) && (curH2 != nullptr))
        if (curH1->degree > curH2->degree) {
            curRes->sibling = curH1;
            curRes = curH1;
            curH1 = curH1->sibling;
        } else {
            curRes->sibling = curH2;
            curRes = curH2;
            curH2 = curH2->sibling;
        }

    // довставка элементов, когда одна из куч закончилась
    if (curH1 == nullptr)
        while (curH2 != nullptr) {
            curRes->sibling = curH2;
            curH2 = curH2->sibling;
            curRes = curRes->sibling;
        }
    else
        while (curH1 != nullptr) {
            curRes->sibling = curH1;
            curH1 = curH1->sibling;
            curRes = curRes->sibling;
        }

    // слияние куч
    bool fl = true;
    while (fl){
        fl = false;
        curRes = result.sibling;
        while ((curRes != nullptr) && (curRes->sibling != nullptr)) {
            if (curRes->degree == curRes->sibling->degree) {
                auto h1 = curRes, h2 = curRes->sibling;
                auto tmp = h1->child;
                auto degree = curRes->degree + curRes->sibling->degree;
                fl = true;
                h1->child = h2;
                h2->parent = h1;
                h1->sibling = h2->sibling;
                h2->sibling = tmp;
                if (h1->key > h2->key) {
                    std::swap(h1->data, h2->data);
                    std::swap(h1->key, h2->key);
                    std::swap(h1->degree, h2->degree);
                }
                curRes->degree = degree;
            } else
                curRes = curRes->sibling;
        }
    }
    return result.sibling;
}

Извлечение минимума

In [6]:
template <typename T>
BinomialNode<T>* extractMin(BinomialNode<T>* root){
    if (root == nullptr)
        return nullptr;
    if (root->child == nullptr)
        return nullptr;

    float best = root->child->key;

    BinomialNode<T>* x = root->child;
    BinomialNode<T>* xBefore = nullptr;
    BinomialNode<T>* curx = root->child;
    BinomialNode<T>* curxBefore = nullptr;
    while (curx != nullptr) {
        if (curx->key < best) {
            best = curx->key;
            x = curx;
            xBefore = curxBefore;
        }
        curxBefore = curx;
        curx = curx->sibling;
    }
    if (xBefore == nullptr)
        root->child = x->sibling;
    else
        xBefore->sibling = x->sibling;

    BinomialNode<T> temp_heap;

    temp_heap.sibling = x->child;

    root->child = mergeBinomialHeaps(root->child, temp_heap.sibling);
    return x;
}

Уменьшение ключа

In [7]:
template <typename T>
void decreaseKey(BinomialNode<T>* root, BinomialNode<T>* x, float key){
    if (key > x->key) return;
    x->key = key;
    auto y = x;
    auto z = y->parent;
    while ((z != root) and (y->key < z->key)){
        std::swap(y->key, z->key);
        std::swap(y->data, z->data);
        y = z;
        z = y->parent;
    }
}

Удаление узла

In [8]:
template <typename T>
void deleteNode(BinomialNode<T>* root, BinomialNode<T>* node){
    decreaseKey(root, node, -1);
    auto tmp = extractMin(root);
    delete tmp;
}

Конструктор

In [9]:
template <typename T>
BinomialHeap<T>::BinomialHeap(T invalidValue) {
    //root = nullptr;
    invalid = invalidValue;
    root.child = nullptr;
    root.sibling = nullptr;
    root.sibling = nullptr;
    root.degree = 0;
    root.key = 0.0;
}

Деструктор

In [10]:
template <typename T>
BinomialHeap<T>::~BinomialHeap() {
    std::stack<BinomialNode<T>*> stack;
    stack.push(root.child);

    while (true){
        if (stack.empty())
            break;

        auto curr = stack.top(); stack.pop();

        if (curr != nullptr){
            stack.push(curr->sibling);
            stack.push(curr->child);
            delete curr;
        }
    }
}

input_line_16:3:10: error: no member named 'stack' in namespace 'std'
    std::stack<BinomialNode<T>*> stack;
    ~~~~~^
input_line_16:3:31: error: expected '(' for function-style cast or type construction
    std::stack<BinomialNode<T>*> stack;
               ~~~~~~~~~~~~~~~^
input_line_16:3:32: error: expected expression
    std::stack<BinomialNode<T>*> stack;
                               ^
input_line_16:3:34: error: use of undeclared identifier 'stack'
    std::stack<BinomialNode<T>*> stack;
                                 ^
input_line_16:4:5: error: use of undeclared identifier 'stack'
    stack.push(root.child);
    ^
input_line_16:6:13: error: use of undeclared identifier 'stack'
        if (stack.empty())
            ^
input_line_16:8:21: error: use of undeclared identifier 'stack'
        auto curr = stack.top(); stack.pop();
                    ^
input_line_16:8:34: error: use of undeclared identifier 'stack'
        auto curr = stack.top(); stack.pop();
                   

Interpreter Error: 

Вставка

In [ ]:
template <typename T>
void BinomialHeap<T>::insert(T data, float key){
    auto node = newNodeInit(data, key);
    root.child = mergeBinomialHeaps(root.child, node);
    root.child->parent = &root;
}

Получение минимума

In [ ]:
доделать


template <typename T>
T BinomialHeap<T>::getMinimum(T data)  {
    return root.child->data;
}

Извлечение минимума

In [ ]:
template <typename T>
T BinomialHeap<T>::extractMin() {
    auto min = ::extractMin(&root);
    if (min != nullptr) {
        T data = min->data;
        delete min;
        return data;
    }
    return invalid;
}

Слияние

In [ ]:
template <typename T>
void BinomialHeap<T>::merge(BinomialHeap<T>* heap){
    root.child = mergeBinomialHeaps(root.child, heap->root.child);
}

Уменьшение ключа

In [ ]:
template <typename T>
void BinomialHeap<T>::decreaseKey(BinomialNode<T>* node, float key) {
    decreaseKey(&root, key);
}

Удаление

In [ ]:
template <typename T>
void BinomialHeap<T>::remove(BinomialNode<T>* node){
    deleteNode(&root, node);
}

## Левацкая куча

In [ ]:
template <typename T>
struct LeftistNode
{
    float key;
    int dist;
    LeftistNode *left, *right, *parent;
    T data;
};

In [ ]:
template <typename T>
class LeftistHeap {
public:
    LeftistNode<T>* root;
    T invalid;

    LeftistHeap(T invalidValue);
    virtual ~LeftistHeap();

    void merge(LeftistHeap<T>* heap);
    void insert(T data, float key);
    T getMinimum();
    T extractMin();
    void decreaseKey(LeftistNode<T>* node, float key = -1);
    void remove(LeftistNode<T>* node);

};

In [ ]:
template <typename T>
int getDist(LeftistNode<T>* node){
    if (node == nullptr){
        return -1;
    }else{
        return node->dist;
    }
}


In [ ]:

template <typename T>
LeftistNode<T>* newLeftistNode(){
    auto result = new LeftistNode<T>;
    result->left = nullptr;
    result->right = nullptr;
    result->parent = nullptr;
    result->dist = 0;
    result->key = 0.0;
    return result;
}

In [ ]:
template <typename T>
LeftistNode<T>* newLeftistNodeInit(T data, float key = 0.0){
    LeftistNode<T>* result = newLeftistNode<T>();
    result->data = data;
    result->key = key;
    return result;
}

In [ ]:
template <typename T>
LeftistNode<T>* merge(LeftistNode<T>* a, LeftistNode<T>* b){
    if (a == nullptr)
        return b;
    if (b == nullptr)
        return  a;
    if(a->key > b->key)
        swap(a, b);

    a->right = merge(a->right, b);
    a->right->parent = a;

    if (a->left == nullptr) {
        swap(a->left, a->right);
        a->dist = 0; //ToDo
    } else if (a->right and a->left and (getDist(a->right) > getDist(a->left))){
        swap(a->right, a->left);
    }

    a->dist = min(getDist(a->right), getDist(a->left) )+ 1;
    return a;
}

In [ ]:
template <typename T>
LeftistNode<T>* decreaseKey(LeftistNode<T>* node, LeftistNode<T>* root, float key){
    if (key < node->key)
        node->key = key;
    else
        return root;

    auto cur = node->parent;
    if (cur->left == node)
        cur->left = nullptr;
    else
        cur->right = nullptr;
    node->parent = nullptr;
    cur->dist = 0;

    while (cur != root){
        if ((cur->left == nullptr) or (cur->right == nullptr))
            cur->dist = 0;
        else
            cur->dist = min(cur->left->dist, cur->right->dist) + 1;

        if (cur->right->dist > cur->left->dist){
            auto tmp = cur->right;
            cur->right = cur->left;
            cur->left = tmp;
        }

        cur  = cur->parent;
    }

    //root = merge(root, node->left);
    //root = merge(root, node->right);
    root = merge(root, node);
    return root;
}

In [ ]:
template <typename T>
LeftistHeap<T>::LeftistHeap(T invalidValue) {
    root = nullptr;
    invalid = invalidValue;
}

In [ ]:
template <typename T>
LeftistHeap<T>::~LeftistHeap() {
    if(root == nullptr)
        return;

    std::stack<LeftistNode<T>*> stack;
    stack.push(root);

    while (!stack.empty()){

        auto curr = stack.top(); stack.pop();

        if (curr != nullptr){
            stack.push(curr->left);
            stack.push(curr->right);
            delete curr;
        }
    }

}

In [ ]:
template <typename T>
void LeftistHeap<T>::merge(LeftistHeap<T>* heap){
    root = merge(root, heap->root);
}

In [ ]:
template <typename T>
void LeftistHeap<T>::insert(T data, float key){
    auto tmp = newLeftistNodeInit<T>(data, key);
    root = merge(root, tmp);
}


In [ ]:
template <typename T>
T LeftistHeap<T>::getMinimum(){
    if (root != nullptr)
        return root->data;
    return invalid;
}

In [ ]:
template <typename T>
T LeftistHeap<T>::extractMin(){
    auto best = getMinimum();
    if (best != invalid) {
        auto tmp = merge(root->left, root->right);
        delete root;
        root = tmp;
        //root = ::merge(root->left, root->right);
    }
    return best;
}

In [ ]:
template <typename T>
void LeftistHeap<T>::decreaseKey(LeftistNode<T>* node, float key){
    root = ::decreaseKey(node, root, key);
}

In [ ]:
template <typename T>
void LeftistHeap<T>::remove(LeftistNode<T>* node){
    //node->key = -1; // минус бесконечность.
    decreaseKey(node, -1);
    auto tmp = merge(root->left, root->right);
    delete root;
    root = tmp;
}


## Косая куча

In [ ]:
template <typename T>
struct SkewNode
{
    float key;
    SkewNode *left, *right, *parent;
    T data;
};

In [ ]:
template<typename T>
class SkewHeap {
private:
    SkewNode<T>* root;
    T invalid;
public:
    SkewHeap(T invalidValue);
    virtual ~SkewHeap();

    void merge(SkewHeap<T>* heap);
    void insert(T data, float key);
    T getMinimum();
    T extractMin();
};

In [ ]:
template <typename T>
SkewNode<T>* newSkewNode(){
    auto result = new SkewNode<T>;
    result->left = nullptr;
    result->right = nullptr;
    result->parent = nullptr;
    result->dist = 0;
    result->key = 0.0;
    return result;
}

In [ ]:
template <typename T>
SkewNode<T>* newSkewNodeInit(T data, float key = 0.0){
    SkewNode<T>* result = newSkewNode<T>();
    result->data = data;
    result->key = key;
    return result;
}


In [ ]:
template<class T>
SkewNode<T>* merge(SkewNode<T>* root_1, SkewNode<T>* root_2)
{
    SkewNode<T>* firstRoot = root_1;
    SkewNode<T>* secondRoot = root_2;

    if (firstRoot == nullptr)
        return secondRoot;

    else if (secondRoot == nullptr)
        return firstRoot;

    if (firstRoot->key < secondRoot->key)
    {
        SkewNode<T>* tempHeap = firstRoot->right;
        firstRoot->right = firstRoot->left;
        firstRoot->left = merge(secondRoot, tempHeap);
        return firstRoot;
    }
    else
        return merge(secondRoot, firstRoot);
}

In [ ]:
template<typename T>
SkewHeap<T>::SkewHeap(T invalidValue) {
    root = nullptr;
    invalid = invalidValue;
}

In [ ]:
template<typename T>
SkewHeap<T>::~SkewHeap() {
    if(root == nullptr)
        return;

    std::stack<SkewNode<T>*> stack;
    stack.push(root);

    while (!stack.empty()){

        auto curr = stack.top(); stack.pop();

        if (curr != nullptr){
            stack.push(curr->left);
            stack.push(curr->right);
            delete curr;
        }
    }

}

In [ ]:
template<typename T>
void SkewHeap<T>::merge(SkewHeap<T>* heap){
    root = merge(root, heap->root);
}

In [ ]:
template<typename T>
void SkewHeap<T>::insert(T data, float key){
    auto tmp = newSkewNodeInit(data, key);
    root = merge(root, tmp);
}

In [ ]:
template<typename T>
T SkewHeap<T>::getMinimum(){
    if (root == nullptr)
        return invalid;
    return root->data;
}

In [ ]:
template<typename T>
T SkewHeap<T>::extractMin(){
    auto data = getMinimum();
    if (data == invalid)
        return invalid;
    auto tmp = merge(root->left, root->right);
    delete root;
    root = tmp;
    return  data;
}

## Фиббоначева куча

Амортизированная сложность операций:
* add O(1)
* merge O(1)
* getBest O(1)
* deleteBest O(logN)
* changeKey O(logN)

In [ ]:
template <typename DataType, typename KeyType>
struct FibonacciNode{
    KeyType key;      // ключ
    FibonacciNode* parent;  // указатель на родительский узел
    FibonacciNode* child;   // указатель на один из дочерних узлов
    FibonacciNode* prev;    // указатель на левый узел того же предка
    FibonacciNode* next;   // указатель на правый узел того же предка
    int degree;   // степень вершины
    bool mark; // был ли удален в процессе изменения ключа ребенок этой вершины)
    DataType data;
    FibonacciNode(){
        parent = child = prev = next = nullptr;
        degree = 0;
        mark = false;
    }
    FibonacciNode(KeyType Key, DataType Data){
        parent = child = prev = next = nullptr;
        degree = 0;
        mark = false;
        key = Key;
        data = Data;
    }
    ~FibonacciNode(){
        parent = child = prev = next = nullptr;
        degree = 0;
        mark = false;
    }
};


In [ ]:
template <typename DataType, typename KeyType, typename Comporator>
class FibonacciHeap {
private:
    void reset();
    void uniteRings(FibonacciNode<DataType, KeyType>* mainRing, FibonacciNode<DataType, KeyType>* otherRing);
    void consolidate();

    void cut(FibonacciNode<DataType, KeyType>* node);
    void cascadingCut(FibonacciNode<DataType, KeyType>* node);



public:
    FibonacciNode<DataType, KeyType>* best;
    DataType infinity;
    int size;

    FibonacciHeap(DataType MinusInfinity);
    ~FibonacciHeap();
    void merge(FibonacciHeap<DataType, KeyType, Comporator>* heap);
    void insert(DataType data, float key);
    DataType getBest();
    void deleteBest();
    void changeKey(FibonacciNode<DataType, KeyType>* node, KeyType key);
    void remove(FibonacciNode<DataType, KeyType>* node);
};


In [ ]:
template <typename DataType, typename KeyType>
FibonacciNode<DataType, KeyType>* newFibonacciNode(){
    auto res = new FibonacciNode<DataType, KeyType>;
    res->parent = res->child = res->prev = res->next = nullptr;
    res->degree = 0;
    res->mark = false;
    return res;
}


In [ ]:
template <typename DataType, typename KeyType>
FibonacciNode<DataType, KeyType>* initFibonacciNode(DataType Data, KeyType Key){
    auto res = newFibonacciNode<DataType, KeyType>();
    res->key = Key;
    res->data = Data;
    return res;
}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
FibonacciHeap<DataType, KeyType, Comparator>::FibonacciHeap(DataType MinusInfinity){
    infinity = MinusInfinity;
    best = nullptr;
    size = 0;

}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
FibonacciHeap<DataType, KeyType, Comparator>::~FibonacciHeap(){
    if(best == nullptr)
        return;

    stack<FibonacciNode<DataType, KeyType>*> stack;
    stack.push(best);

    while (!stack.empty()){
        auto curr = stack.top(); stack.pop();
        if (curr != nullptr){
            if (curr->prev != nullptr)
                stack.push(curr->prev);
            if (curr->next != nullptr)
                stack.push(curr->next);
            delete curr;
        }
    }
}


In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
void FibonacciHeap<DataType, KeyType, Comparator>::insert(DataType data, KeyType key){

    FibonacciNode<DataType, KeyType>* node = initFibonacciNode(data, key);
    if(size == 0){
        best = node;
        best->next = best;
        best->prev = best;
    } else {
        auto tmp = best->next;
        best->next = node;
        node->next = tmp;
        if (tmp != nullptr)
            tmp->prev = node;
        node->prev = best;
    }
    if (Comparator(node->key, best->key))
        best = node;
    size++;
}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
DataType FibonacciHeap<DataType, KeyType, Comparator>::getBest() {
    if (best == nullptr)
        return infinity;
    return best->data;
}


In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
void FibonacciHeap<DataType, KeyType, Comparator>::reset()  {
    best = nullptr;
    size = 0;
}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
FibonacciNode<DataType, KeyType>* FibonacciHeap<DataType, KeyType, Comparator>::uniteRings(FibonacciNode<DataType, KeyType>* mainRing, FibonacciNode<DataType, KeyType>* otherRing){
    if (otherRing == nullptr)
        return mainRing;
    if (mainRing == nullptr)
        return otherRing;
    auto mainEnd = mainRing->prev;
    auto otherEnd = otherRing->prev;
    mainEnd->next = otherRing;
    otherRing->prev = mainEnd;
    otherEnd->next = mainRing;
    mainRing->prev = otherEnd;
    return mainRing;
}


In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
FibonacciNode<DataType, KeyType>* FibonacciHeap<DataType, KeyType, Comparator>::addTree
                            (FibonacciNode<DataType, KeyType>* Tree1, FibonacciNode<DataType, KeyType>* Tree2){
    if (Tree1 == nullptr)
        return Tree2;
    if (Tree2 == nullptr)
        return Tree1;

    FibonacciNode<DataType, KeyType> *main, *other;
    if(Comparator(Tree1->key, Tree2->key)){
        main = Tree1;
        other = Tree2;
    } else {
        main = Tree2;
        other = Tree1;
    }

    auto prev = other->prev;
    auto next = other->next;

    prev->next = next;
    next->prev = prev;

    other->parent = main;
    if (main->child != nullptr){
        auto tmp = main->child->next;
        main->child->next = other;
        other->next = tmp;
        tmp->prev = other;
        other->prev = main->child;
    } else {
        main->child = other;
        other->next = other;
        other->prev = other;
    }
    main->degree++;
    return main;
}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
void FibonacciHeap<DataType, KeyType, Comparator>::merge(FibonacciHeap<DataType, KeyType, Comparator>* heap){
    if (heap->size == 0)
        return;
    if (size == 0){
        best = heap->best;
        size = heap->size;
    } else {
        if (Comporator(heap->best->key, best->key))
            best = heap->best;
        best = uniteRings(best, heap->best);
        size += heap->size;
        heap->reset();
    }
}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
void FibonacciHeap<DataType, KeyType, Comparator>::consolidate(){
    FibonacciNode<DataType, KeyType>* map[65] = {nullptr};
    map[best->degree] = best;
    auto cur = best->next;
    while (cur != map[cur->degree]){ //???
        if(!map[cur->degree]){
            map[cur->degree] = cur;
            cur = cur->next;
        }else{
            auto degree = cur->degree;
            auto conflict = map[degree];
            FibonacciNode<DataType, KeyType> *main, *other;
            cur = addTree(conflict, cur);
            map[degree] = nullptr;
        }
        if(Comparator(cur->key, best->key))
            best = cur;
    }
}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
void FibonacciHeap<DataType, KeyType, Comparator>::deleteBest(){
    if (best == nullptr)
        return;
    auto tmpBest = best;
    best = uniteRings(best, best->child);
    if (size == 1) {
        delete best;
        best = nullptr;
        size = 0;
        return;
    }
    auto prev = best->prev;
    auto next = best->next;
    prev->next = next;
    next->prev = prev;

    best = best->next;
    consolidate();

    delete tmpBest;
    size--;
}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
DataType FibonacciHeap<DataType, KeyType, Comparator>::extractBest(){
    auto result = getBest();
    deleteBest();
    return result;
}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
void FibonacciHeap<DataType, KeyType, Comparator>::cut(FibonacciNode<DataType, KeyType>* node){
    auto prev = node->prev;
    auto next = node->next;
    next->prev = prev;
    prev->next = next;
    node->parent->degree--;
    if (node->parent->child == node)
        if (node->next == node)
            node->parent->child = nullptr;
        else
            node->parent->child = node->next;
    node->next = node;
    node->prev = node;
    node->parent = nullptr;
    best = uniteRings(best, node);

}


In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
void FibonacciHeap<DataType, KeyType, Comparator>::cascadingCut(FibonacciNode<DataType, KeyType>* node){
    while(node->mark){
        cut(node);
        node = node->parent;
    }
    node->mark = true;
}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
void FibonacciHeap<DataType, KeyType, Comparator>::changeKey(FibonacciNode<DataType, KeyType>* node, KeyType key){
    if (Comparator(node->parent->key, key)){
        node->key = key;
        return;
    }
    auto parent = node->parent;
    cut(node);
    cascadingCut(parent);
}

In [ ]:
template <typename DataType, typename KeyType, bool (*Comparator)(DataType a, DataType b)>
void FibonacciHeap<DataType, KeyType, Comparator>::remove(FibonacciNode<DataType, KeyType>* node){
    changeKey(node, infinity);
    deleteBest();
}

## Толстая куча

## Тонкая куча

## Толстая куча на избыточном счетчике

## Куча Бродала-Окасаки